In [1]:
import glob
import itertools
import json
import os
import sys

import tqdm
from tensorflow.keras.models import load_model

from classes.VQVAE import VQVAE2
from classes.PixelCNN2 import ConditionalPixelCNN2
from utils.callbacks import WandbImagesVQVAE, Save_VQVAE_Weights, Save_PixelCNN_Weights, Save_VQVAE2_Weights, \
    WandbImagesVQVAE2
from utils.functions import map_vqvae2_weights

import tensorflow as tf
from tensorflow import keras
import numpy as np
import wandb
from wandb.keras import WandbCallback
from tensorflow.data import AUTOTUNE


from os.path import join as opj
from imutils import paths
import tqdm
import glob
import json

In [2]:
#set the first GPU
os.environ["CUDA_VISIBLE_DEVICES"]="0"

print(os.environ.get("CUDA_VISIBLE_DEVICES"))

0


In [3]:
wandb.login()

phase="VQ-VAE2_Training"
#phase="PixelCNN_Training"

config={"dataset":"celebA", "type":"VQ-VAE2","phase":phase}

images_dir=r"C:\Users\matte\Dataset\tor_vergata\Dataset\Img\img_align_celeba" #local
images_dir=r"/home/matteo/NeuroGEN/Dataset/Img/img_align_celeba"

wandb: Currently logged in as: matteoferrante (use `wandb login --relogin` to force relogin)


In [10]:
BS = 64
EPOCHS=40
INIT_LR=1e-4

config["BS"]=BS
config["EPOCHS"]=EPOCHS
config["INIT_LR"]=INIT_LR

## Dataloaders

In [11]:
def load_images(imagePath):
    # read the image from disk, decode it, resize it, and scale the
    # pixels intensities to the range [0, 1]
    image = tf.io.read_file(imagePath)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, (128, 128)) / 255.0

    #eventually load other information like attributes here
    
    # return the image and the extra info
    
    
    return image

In [14]:
print("[INFO] loading image paths...")
imagePaths = list(paths.list_images(images_dir))


train_len=int(0.8*len(imagePaths))
val_len=int(0.1*len(imagePaths))
test_len=int(0.1*len(imagePaths))

train_imgs=imagePaths[:train_len]                                #      80% for training
val_imgs=imagePaths[train_len:train_len+val_len]                 #      10% for validation
test_imgs=imagePaths[train_len+val_len:]                         #      10% for testing

print(f"[TRAINING]\t {len(train_imgs)}\n[VALIDATION]\t {len(val_imgs)}\n[TEST]\t\t {len(test_imgs)}")

[INFO] loading image paths...
[TRAINING]	 0
[VALIDATION]	 0
[TEST]		 0


In [17]:
#TRAINING 

train_dataset = tf.data.Dataset.from_tensor_slices(train_imgs)
train_dataset = (train_dataset
    .shuffle(1024)
    .map(load_images, num_parallel_calls=AUTOTUNE)
    .cache()
    .repeat()
    .batch(BS)
    .prefetch(AUTOTUNE)
)

ts=len(train_imgs)//BS

##VALIDATION

val_dataset = tf.data.Dataset.from_tensor_slices(val_imgs)
val_dataset = (val_dataset
    .shuffle(1024)
    .map(load_images, num_parallel_calls=AUTOTUNE)
    .cache()
    .repeat()
    .batch(BS)
    .prefetch(AUTOTUNE)
)

vs=len(val_imgs)//BS

## TEST

test_dataset = tf.data.Dataset.from_tensor_slices(test_imgs)
test_dataset = (test_dataset
    .shuffle(1024)
    .map(load_images, num_parallel_calls=AUTOTUNE)
    .cache()
    .batch(BS)
    .prefetch(AUTOTUNE)
)

TypeError: in user code:

    File "C:\Users\matte\AppData\Local\Temp/ipykernel_14824/1628486324.py", line 4, in load_images  *
        image = tf.io.read_file(imagePath)

    TypeError: Input 'filename' of 'ReadFile' Op has type float32 that does not match expected type of string.


## Model definition

In [4]:
print(f"[INFO] Training VQ_VAE Model")

input_shape=(128,128,3)
latent_dim=256
num_embeddings=1024


config["latent_dim"]=latent_dim
config["num_embeddings"]=num_embeddings
config["input_shape"]=input_shape

g=VQVAE2(input_shape,latent_dim=latent_dim,num_embeddings=num_embeddings,train_variance=4,n_res_channel=latent_dim,channels=128)




[INFO] Training VQ_VAE Model


In [5]:
print(g.encoder_b.summary())

print(g.encoder_t.summary())
print(g.conditional_bottom.summary())

print(g.decoder.summary())

Model: "bottom_encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 64, 64, 128)       6272      
                                                                 
 activation (Activation)     (None, 64, 64, 128)       0         
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 128)       262272    
                                                                 
 activation_1 (Activation)   (None, 32, 32, 128)       0         
                                                                 
 conv2d_2 (Conv2D)           (None, 16, 16, 128)       262272    
                                                                 
 activation_2 (Activation)   (None, 16, 16, 128)    

## Phase 1: Train the VQ VAE 2

In [ ]:
wandb.init(project="TorVergataExperiment-Generative",config=config)

In [ ]:
os.makedirs("models/vqvae2_celeba",exist_ok=True)


model_check= Save_VQVAE2_Weights("models/vqvae2_celeba")



es=tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=3,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=True,
)


callbacks=[
    WandbImagesVQVAE2(test_dataset,sample=False),
    WandbCallback(),
    model_check,
    es,
]

In [ ]:
g.compile(keras.optimizers.Adam(INIT_LR))

In [ ]:
g.fit(train_dataset,validation_data=val_dataset,steps_per_epoch=ts,validation_steps=vs,epochs=20,callbacks=callbacks)

g.save_dict("models/vqvae2_celeba/dict.json")
g.vqvae.save_weights("models/vqvae2_celeba/model_vqvae2_weights_cifar10.h5")
g.vqvae.save("models/vqvae2_celeba/model_vqvae2_model_cifar10.h5")